In [2]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Building out the Dataframes

In [9]:
SQL = '''SELECT * from Transfers247 where playername like 'vandarius%' '''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
vandariuscowan_alabama_2017,vandariuscowan_alabama,vandariuscowan,2017,LB,2


In [10]:
df_247.loc['vandariuscowan_alabama_2017']

ID                      vandariuscowan_alabama
PlayerName                      vandariuscowan
Year                                      2017
StandardizedPosition                        LB
KeyPositionGroup                             2
Name: vandariuscowan_alabama_2017, dtype: object

In [11]:
SQL = '''SELECT * from UnlinkedNCAA '''
df_ncaa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

,PlayerName,Year,StandardizedPosition,KeyPositionGroup
NCAA_ID,,,,
aadarkwah_massachusetts,aadarkwah,2017,WR,1
aadreekisconner_mississippistate,aadreekisconner,2019,DB,2
aarenvaughns_utahstate,aarenvaughns,2017,WR,1
aarionharvey_centralmichigan,aarionharvey,2020,OL,1
aaronabbott_easternmichigan,aaronabbott,2015,DB,2
...,...,...,...,...
zionjohnson_bostoncollege,zionjohnson,2019,OL,1
zionjohnson_louisville,zionjohnson,2019,OL,1
zionkelly_airforce,zionkelly,2020,DB,2


In [7]:
df_ncaa.loc['vandariuscowan_westvirginia']

PlayerName              vandariuscowan
Year                              2019
StandardizedPosition                LB
KeyPositionGroup                     2
Name: vandariuscowan_westvirginia, dtype: object

In [18]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [19]:
pairs = indexer.index(df_247, df_ncaa)

In [20]:
from functions import YearNFL

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')


In [21]:
features = c.compute(pairs, df_247, df_ncaa)

In [22]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  \
247_IDYR                    NCAA_ID                                   
vandariuscowan_alabama_2017 aaronabbott_easternmichigan    0.214286   
                            aaronbrown_louisianatech       0.428571   
                            aaroncolvin_baylor             0.357143   
                            aaroncurry_northwestern        0.285714   
                            aaroncurry_tcu                 0.285714   
...                                                             ...   
                            zekeedmonds_massachusetts      0.071429   
                            zekeriser_maryland             0.214286   
                            zenochoi_hawaii                0.142857   
                            zepheniahgrimes_illinois       0.066667   
                            zhenderson_texasam             0.357143   

                                                         StandardizedPosition  \
247_IDYR                    NCAA_ID                                             
vandariuscowan_alabama_2017 aaronabbott_easternmichigan                     0   
                            aaronbrown_louisianatech                        0   
                            aaroncolvin_baylor                              0   
                            aaroncurry_northwestern                         0   
                            aaroncurry_tcu                                  0   
...                                                                       ...   
                            zekeedmonds_massachusetts                       1   
                            zekeriser_maryland                              0   
                            zenochoi_hawaii                                 0   
                            zepheniahgrimes_illinois                        1   
                            zhenderson_texasam                              0   

                                                         KeyPositionGroup  \
247_IDYR                    NCAA_ID                                         
vandariuscowan_alabama_2017 aaronabbott_easternmichigan                 1   
                            aaronbrown_louisianatech                    1   
                            aaroncolvin_baylor                          1   
                            aaroncurry_northwestern                     1   
                            aaroncurry_tcu                              1   
...                                                                   ...   
                            zekeedmonds_massachusetts                   1   
                            zekeriser_maryland                          1   
                            zenochoi_hawaii                             1   
                            zepheniahgrimes_illinois                    1   
                            zhenderson_texasam                          1   

                                                         Year       sum  
247_IDYR                    NCAA_ID                                      
vandariuscowan_alabama_2017 aaronabbott_easternmichigan   1.0  0.553571  
                            aaronbrown_louisianatech      1.0  0.607143  
                            aaroncolvin_baylor            1.0  0.589286  
                            aaroncurry_northwestern       1.0  0.571429  
                            aaroncurry_tcu                1.0  0.571429  
...                                                       ...       ...  
                            zekeedmonds_massachusetts     1.0  0.767857  
                            zekeriser_maryland            1.0  0.553571  
                            zenochoi_hawaii               1.0  0.535714  
                            zepheniahgrimes_illinois      1.0  0.766667  
                            zhenderson_texasam            1.0  0.589286  

[2560 rows x 5 columns]

In [24]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
#    data = data.loc[data['sum'].idxmax()]
#    if (data['sum'] > .8):
    filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")